In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv("spam.csv",encoding='latin-1')

In [3]:
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [4]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [5]:
df

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [6]:
 df = df.rename(columns={'v1': 'label', 'v2': 'message'})

In [7]:
df

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   label    5572 non-null   object
 1   message  5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [9]:
df.isnull().sum()

label      0
message    0
dtype: int64

In [10]:
df.duplicated().sum()

403

In [11]:
df.drop_duplicates(inplace=True)

In [12]:
df.reset_index(drop=True, inplace=True)

In [14]:
df.shape

(5169, 2)

In [16]:
import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Anish\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Anish\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
 stop_words_list = stopwords.words('english')
stop_words = set(stop_words_list)
def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)
    words = nltk.word_tokenize(text)
    processed_words = [
        word
        for word in words
        if word not in stop_words and len(word) > 2 
    ]
    return ' '.join(processed_words)

In [19]:
if 'message' in df.columns:
    df['processed_message'] = df['message'].apply(preprocess_text)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5169 entries, 0 to 5168
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   label              5169 non-null   object
 1   message            5169 non-null   object
 2   processed_message  5169 non-null   object
dtypes: object(3)
memory usage: 121.3+ KB


In [21]:
df.head()

,label,message,processed_message
0,ham,"Go until jurong point, crazy.. Available only ...",jurong point crazy available bugis great world...
1,ham,Ok lar... Joking wif u oni...,lar joking wif oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry wkly comp win cup final tkts may te...
3,ham,U dun say so early hor... U c already then say...,dun say early hor already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah dont think goes usf lives around though


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(
    max_features=3000,
    ngram_range=(1, 2)
)

print("\nApplying TF-IDF Vectorizer...")
if 'label_num' not in df.columns:
    print("Warning: 'label_num' column not found. Re-creating from 'label'...")
    df['label_num'] = df['label'].map({'ham': 0, 'spam': 1})

X = tfidf_vectorizer.fit_transform(df['processed_message'])
y = df['label_num'] # Use the numerical label column

print("TF-IDF Vectorization complete.")
print("Shape of TF-IDF matrix (features X):", X.shape)
print("Shape of target variable (y):", y.shape)
feature_names = tfidf_vectorizer.get_feature_names_out()
print("\nExample Feature Names (Vocabulary):", feature_names[1000:1010])


Applying TF-IDF Vectorizer...
TF-IDF Vectorization complete.
Shape of TF-IDF matrix (features X): (5169, 3000)
Shape of target variable (y): (5169,)

Example Feature Names (Vocabulary): ['going home' 'going join' 'going lunch' 'going rain' 'going sch'
 'going sleep' 'going take' 'gon' 'gon call' 'gon get']


In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    random_state=42,
    stratify=y
)

print("\nData Splitting Complete.")
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

print("\nTraining Set Label Distribution:")
print(y_train.value_counts(normalize=True)) 

print("\nTest Set Label Distribution:")
print(y_test.value_counts(normalize=True))


Data Splitting Complete.
Shape of X_train: (4135, 3000)
Shape of y_train: (4135,)
Shape of X_test: (1034, 3000)
Shape of y_test: (1034,)

Training Set Label Distribution:
label_num
0    0.873761
1    0.126239
Name: proportion, dtype: float64

Test Set Label Distribution:
label_num
0    0.873308
1    0.126692
Name: proportion, dtype: float64


In [24]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

nb_model = MultinomialNB()
lr_model = LogisticRegression(max_iter=1000, random_state=42) 
svm_model = SVC(random_state=42, probability=True) 

print("\nTraining Multinomial Naive Bayes...")
nb_model.fit(X_train, y_train)
print("Naive Bayes training complete.")

print("\nTraining Logistic Regression...")
lr_model.fit(X_train, y_train)
print("Logistic Regression training complete.")

print("\nTraining SVM...")
svm_model.fit(X_train, y_train)
print("SVM training complete.")

models = {
    "Naive Bayes": nb_model,
    "Logistic Regression": lr_model,
    "SVM": svm_model
}
print("\nAll models trained and stored.")


Training Multinomial Naive Bayes...
Naive Bayes training complete.

Training Logistic Regression...
Logistic Regression training complete.

Training SVM...
SVM training complete.

All models trained and stored.


In [25]:
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                           f1_score, confusion_matrix, classification_report)

results = {}

print("\n--- Model Evaluation on Test Set ---")
for name, model in models.items():
    print(f"\n----- Evaluating {name} -----")
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, pos_label=1)
    recall = recall_score(y_test, y_pred, pos_label=1)
    f1 = f1_score(y_test, y_pred, pos_label=1)
    conf_matrix = confusion_matrix(y_test, y_pred)

    results[name] = {'Accuracy': accuracy, 'Precision (Spam)': precision,
                     'Recall (Spam)': recall, 'F1-Score (Spam)': f1}

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision (Spam): {precision:.4f}")
    print(f"Recall (Spam): {recall:.4f}")
    print(f"F1-Score (Spam): {f1:.4f}")

    print("\nConfusion Matrix:")
    print(conf_matrix)

    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=['Ham (0)', 'Spam (1)']))
    print("-" * 40)

print("\n--- Summary of Results ---")
results_df = pd.DataFrame(results).T
print(results_df)


--- Model Evaluation on Test Set ---

----- Evaluating Naive Bayes -----
Accuracy: 0.9710
Precision (Spam): 0.9903
Recall (Spam): 0.7786
F1-Score (Spam): 0.8718

Confusion Matrix:
[[902   1]
 [ 29 102]]

Classification Report:
              precision    recall  f1-score   support

     Ham (0)       0.97      1.00      0.98       903
    Spam (1)       0.99      0.78      0.87       131

    accuracy                           0.97      1034
   macro avg       0.98      0.89      0.93      1034
weighted avg       0.97      0.97      0.97      1034

----------------------------------------

----- Evaluating Logistic Regression -----
Accuracy: 0.9478
Precision (Spam): 0.9639
Recall (Spam): 0.6107
F1-Score (Spam): 0.7477

Confusion Matrix:
[[900   3]
 [ 51  80]]

Classification Report:
              precision    recall  f1-score   support

     Ham (0)       0.95      1.00      0.97       903
    Spam (1)       0.96      0.61      0.75       131

    accuracy                           0.9